In [33]:
from readability import Readability
import LIWC_spanish as LIWC
import nltk
nltk.download('vader_lexicon')  
import numpy as np
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
from collections import Counter
import math
import re
import pandas as pd

import requests
req = requests.get('https://static-content.springer.com/esm/art%3A10.3758%2Fs13428-015-0700-2/MediaObjects/13428_2015_700_MOESM1_ESM.csv')
anew = req.text.split('\n')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/sarmad/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [34]:
from unidecode import unidecode

candidates = {
    'Myriam Bregman' :    ['MYRIAM TERESA BREGMA', 'Myriam Bregman', 'Miriam Bregman', 'Miriam Breman', 'Myriam Breman', 'Myriam', 'Miriam',],
    'Gabriel Solano' :    ['GABRIEL ESTEBAN SOLANO', 'Gabriel Solano', 'Solano',],
    'Alejandro Biondini' :    ['ALEJANDRO CESAR BIONDINI', 'Alejandro Biondini', 'Biondini',],
    'Juan Schiaretti' :    ['JUAN SCHIARETTI', 'Juan Schiaretti', 'Juan Schriareti', 'Schiareti', 'Squiareti',],
    'Horacio Rodriguez Larreta'  :    ['HORACIO ANTONIO RODRIGUEZ LARRETA', 'Horacio Rodriguez Larreta', 'Larreta', 'Lareta', 'Sombrilla', 'Pelado',],
    'Patricia Bullrich' :    ['PATRICIA BULLRICH', 'Patricia Bullrich', 'Bullrich', 'Patricia Bulrich', 'Bulrich', 'Patricia', 'Pato', 'Montonera',],
    'Javier Milei' :    ['JAVIER GERARDO MILEI', 'Javier Milei', 'Miley', 'Mi ley', 'Milaw', 'Leon', 'Peluca',],
    'Nazareno Etchepare' :    ['NAZARENO GABRIEL ETCHEPARE', 'Nazareno Etchepare', 'Etchepare',],
    'Ramiro Vasena' :    ['RAMIRO EDUARDO VASENA', 'Ramiro Vasena', 'Ramiro basena', 'Basena', 'Vasena',],
    'Manuela Castañeira' :    ['MANUELA JIMENA CASTAÑEIRA', 'Manuela Castañeira', 'Castañeira', 'Manuela Castanieira', 'Castanieira',],
    'Raul Albarracin' :    ['RAUL HUMBERTO ALBARRACIN', 'Raul Albarracin', 'Albarracin', 'Albaracin',],
    'Raul Castells' :    ['RAUL ANIBAL CASTELLS', 'Raul Castells', 'Raul Castels', 'Castells', 'Castels',],
    'Santiago Cuneo' :    ['GABRIEL SANTIAGO CÚNEO', 'Santiago Cuneo', 'Cuneo',],
    'Jesus Escobar' :    ['JESUS ARNALDO ESCOBAR', 'Jesus Escobar', 'Escobar',],
    'Marcelo Ramal' :    ['MARCELO ARMANDO RAMAL', 'Marcelo Ramal', 'Ramal',],
    'Guillermo Moreno' :    ['MARIO GUILLERMO MORENO', 'Guillermo Moreno', 'Moreno',],
    'Oscar Giardinelli' :    ['OSCAR ALFREDO RAMON GIARDINELLI', 'Oscar Giardinelli', 'Oscar Giardineli', 'Giardinelli', 'Giardineli',],
    'Martin Ayerbe Ortiz'  :    ['MARTIN MIGUEL AYERBE ORTIZ', 'Martin Ayerbe Ortiz', ],
    'Reina Ibañez' :    ['REINA XIOMARA IBAÑEZ', 'Reina Ibañez', 'Reina Xiomara Ibañez', ],
    'Andres Passamonti' :    ['ANDRES GABRIEL PASSAMONTI', 'Andres Passamonti', 'Passamonti', 'Pasamonti', ],
    'Sergio Tomas Massa'  :    ['SERGIO TOMAS MASSA', 'Sergio Tomas Massa', 'Sergio Massa', 'Sergio Masa', 'Masa', 'Massa', 'Panqueque'],
    'Juan Grabois' :    ['JUAN GRABOIS', 'Juan Grabois', 'Grabois', 'Gravois', ],
    'Julio Barbaro' :    ['JULIO DONATO BARBARO', 'Julio Barbaro', 'Barbaro', ],
    'Eliodoro Martinez' :    ['ELIODORO MARTINEZ', 'Eliodoro Martinez', ],
    'Jorge Oliver' :    ['JORGE EDUARDO OLIVER', 'Jorge Oliver', ],
    'Carolina Bartolini' :    ['CARINA PAOLA BARTOLINI', 'Carolina Bartolini', 'Barotilin', ],
    'Paula Arias' :    ['PAULA LORENA ARIAS', 'Paula Arias'],
}

# lowercase and strip any extraspaces from all the candidates values
for candidate in candidates.keys():
    candidates[candidate] = [unidecode(candidate.lower().strip()) for candidate in candidates[candidate]]



In [35]:
anew_dict = {}
for i in range(1,len(anew)):
  row = anew[i].split(',')
  if len(row)>1:
    anew_dict[row[1]]=[float(row[2]),float(row[5])] 

In [36]:
def anew_extract(text,anew_dict):
  words = text.split(' ')
  D = {}
  for w in words:
    if w in anew_dict:
      if 'valence' not in D:
        D['valence']=[]
      if 'arousal' not in D:
        D['arousal']=[]
      D['valence'].append(anew_dict[w][0])
      D['arousal'].append(anew_dict[w][1])
  D_new = {}
  for f in D:
    D_new[f] = np.mean(D[f])
  return(D_new)

In [37]:
def whiteSpacePerChar(text):
    '''
    Total White Space/Total character
    '''
    totChar=len(text)
    totNoSpace=len(text.replace(' ',''))
    totSpace=totChar-totNoSpace

    return(float(totSpace)/float(totChar))

def digitPerChar(text):
    '''
    total digits/character
    '''
    totChar=len(text)
    NoDigit=text
    for i in range(0,10):
        NoDigit=NoDigit.replace(str(i),'')
    totNoDigit=len(NoDigit)
    totDigit=totChar-totNoDigit
    return(float(totDigit)/float(totChar))

def charLen(text):
    return(len(text)/len(text.split(' ')))

def TTR(text):
  return(len(set(text.split(' ')))/len(text.split(' ')))

def honore(text):
    words=text.split(' ')
    unique=Counter(words)
    num=float(len(words))
    cnt=0
    for w in unique:
        if unique[w]==1:
            cnt=cnt+1
    denom=1-(float(cnt)/float(len(unique)))
    R=100*math.log10(num/denom)
    return(R)

def sichel(text):
    words=text.split(' ')
    unique=Counter(words)
    num=float(len(words))
    cnt=0
    for w in unique:
        if unique[w]==2:
            cnt=cnt+1

    S=float(cnt)/float(len(unique))
    return(S)

def brunet(text):
    words=text.split(' ')
    a=0.172
    W= len(words)** (len(set(words)) **a)
    return(W)



def hapaxLegomena(text):
    '''
    unnormalized
    '''
    words=text.split(' ')
    unique=Counter(words)
    cnt=0
    for w in unique:
        if unique[w]==1:
            cnt=cnt+1

    return(cnt/len(words))

def hapaxDislogemna(text):
    '''
    unnormalized
    '''
    words=text.split(' ')
    unique=Counter(words)
    cnt=0
    for w in unique:
        if unique[w]==2:
            cnt=cnt+1

    return(cnt/len(words))

In [38]:
def safe_apply(func, arg):
    try:
        return func(arg)
    except:
        return np.nan

def complexity_index(text):
  R = {}
  R['hapaxDislogemna']= safe_apply(hapaxDislogemna, text)
  R['hapaxLegomena']= safe_apply(hapaxLegomena, text)
  R['brunet']= safe_apply(brunet, text)
  R['sichel']= safe_apply(sichel, text)
  R['honore'] = safe_apply(honore, text)
  R['TTR'] = safe_apply(TTR, text)
  R['chars'] = safe_apply(charLen, text)
  R['digits'] = safe_apply(digitPerChar, text)
  R['whiteSpace'] = safe_apply(whiteSpacePerChar, text)
  return(R)

In [39]:
def readability_index(text):
  R = {}
  try:
    r = Readability(text)

    R['FKI']=r.flesch_kincaid().score
    R['FKR']=r.flesch().score
    R['GunningFog']=r.gunning_fog().score
    R['CLI']=r.coleman_liau().score
    R['DaleChall']=r.dale_chall().score
    R['ARI']=r.ari().score
    R['linsear']=r.linsear_write().score
    R['spache']=r.spache().score
  
  except:
    R['FKI']= np.nan
    R['FKR']= np.nan
    R['GunningFog']= np.nan
    R['CLI']= np.nan
    R['DaleChall']= np.nan
    R['ARI']= np.nan
    R['linsear']= np.nan
    R['spache']= np.nan

  return(R)

In [40]:
df = pd.read_csv('preprocessed_data.csv')
df

,candidate,source,date,text
0,Patricia Bullrich,clarin,2022-10-20,un encuestador k midió a cristina kirchner con...
1,Horacio Rodriguez Larreta,clarin,2022-10-20,un encuestador k midió a cristina kirchner con...
2,Myriam Bregman,clarin,2022-10-20,un encuestador k midió a cristina kirchner con...
3,Sergio Tomas Massa,clarin,2022-10-20,un encuestador k midió a cristina kirchner con...
4,Javier Milei,clarin,2022-10-20,un encuestador k midió a cristina kirchner con...
...,...,...,...,...
135292,Horacio Rodriguez Larreta,youtube,2023-08-01,"agustina kampfer entrevistó a césar biondini, ..."
135293,Patricia Bullrich,youtube,2023-08-01,"agustina kampfer entrevistó a césar biondini, ..."
135294,Alejandro Biondini,youtube,2023-07-31,entrevista a césar biondini en &quot;somos pm&...
135295,Patricia Bullrich,youtube,2023-07-31,entrevista a césar biondini en &quot;somos pm&...


In [46]:
# make a list of all the candidates mentioned in candidates
candidates_list = []
for c in candidates.values():
    candidates_list += c

candidates_list

['myriam teresa bregma',
 'myriam bregman',
 'miriam bregman',
 'miriam breman',
 'myriam breman',
 'myriam',
 'miriam',
 'gabriel esteban solano',
 'gabriel solano',
 'solano',
 'alejandro cesar biondini',
 'alejandro biondini',
 'biondini',
 'juan schiaretti',
 'juan schiaretti',
 'juan schriareti',
 'schiareti',
 'squiareti',
 'horacio antonio rodriguez larreta',
 'horacio rodriguez larreta',
 'larreta',
 'lareta',
 'sombrilla',
 'pelado',
 'patricia bullrich',
 'patricia bullrich',
 'bullrich',
 'patricia bulrich',
 'bulrich',
 'patricia',
 'pato',
 'montonera',
 'javier gerardo milei',
 'javier milei',
 'miley',
 'mi ley',
 'milaw',
 'leon',
 'peluca',
 'nazareno gabriel etchepare',
 'nazareno etchepare',
 'etchepare',
 'ramiro eduardo vasena',
 'ramiro vasena',
 'ramiro basena',
 'basena',
 'vasena',
 'manuela jimena castaneira',
 'manuela castaneira',
 'castaneira',
 'manuela castanieira',
 'castanieira',
 'raul humberto albarracin',
 'raul albarracin',
 'albarracin',
 'albaraci

In [47]:
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True, nb_workers=64)

INFO: Pandarallel will run on 64 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [48]:
# group the data in such a way that every candidate's data is in a single row
df = df.groupby(['candidate', 'source', 'date'])['text'].apply(lambda x: '\n'.join(x)).reset_index()
df

,candidate,source,date,text
0,Alejandro Biondini,c5n,2023-06-27,los precandidatos presidenciales firmarán un c...
1,Alejandro Biondini,c5n,2023-06-29,la interna menos pensada: el verborrágico sant...
2,Alejandro Biondini,c5n,2023-09-05,"quién es lucía montenegro, una de las anfitrio..."
3,Alejandro Biondini,clarin,2017-10-09,"por la foto con biondini, ricardo iorio fue da..."
4,Alejandro Biondini,clarin,2019-08-09,googleó su nombre y descubrió que el domingo e...
...,...,...,...,...
25641,Sergio Tomas Massa,youtube,2023-10-05,rompecabezas - patricia bullrich pugna por ent...
25642,Sergio Tomas Massa,youtube,2023-10-06,el poder económico entre milei y bullrich: jai...
25643,Sergio Tomas Massa,youtube,2023-10-07,bullrich y melconian manipulan las palabras de...
25644,Sergio Tomas Massa,youtube,2023-10-08,javier milei la rompe en la mesa de mirtha leg...


In [49]:
# extract the number of mentions of each candidate in each article
def find_mentions_count(text):
    mentions_count = {}
    for candidate in candidates_list:
        mentions_count[candidate] = len(re.findall(candidate, text))
    return mentions_count

df_temp = df['text'].parallel_apply(lambda x: pd.Series(find_mentions_count(x)))
df_temp.columns = [f'mentions_{col}' for col in df_temp.columns]

# Directly assign to columns in original dataframe
df[df_temp.columns] = df_temp
# df = pd.concat([df, df_temp], axis=1)
print('Done!')
df


/tmp/ipykernel_64769/2182785831.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[df_temp.columns] = df_temp
/tmp/ipykernel_64769/2182785831.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[df_temp.columns] = df_temp
/tmp/ipykernel_64769/2182785831.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = fr

Done!


,candidate,source,date,text,mentions_myriam teresa bregma,mentions_myriam bregman,mentions_miriam bregman,mentions_miriam breman,mentions_myriam breman,mentions_myriam,...,mentions_julio barbaro,mentions_barbaro,mentions_eliodoro martinez,mentions_jorge eduardo oliver,mentions_jorge oliver,mentions_carina paola bartolini,mentions_carolina bartolini,mentions_barotilin,mentions_paula lorena arias,mentions_paula arias
0,Alejandro Biondini,c5n,2023-06-27,los precandidatos presidenciales firmarán un c...,0,2,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
1,Alejandro Biondini,c5n,2023-06-29,la interna menos pensada: el verborrágico sant...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Alejandro Biondini,c5n,2023-09-05,"quién es lucía montenegro, una de las anfitrio...",0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
3,Alejandro Biondini,clarin,2017-10-09,"por la foto con biondini, ricardo iorio fue da...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Alejandro Biondini,clarin,2019-08-09,googleó su nombre y descubrió que el domingo e...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25641,Sergio Tomas Massa,youtube,2023-10-05,rompecabezas - patricia bullrich pugna por ent...,0,9,31,0,0,9,...,0,0,0,0,0,0,0,0,0,0
25642,Sergio Tomas Massa,youtube,2023-10-06,el poder económico entre milei y bullrich: jai...,0,1,33,0,0,1,...,0,0,0,0,0,0,0,0,0,0
25643,Sergio Tomas Massa,youtube,2023-10-07,bullrich y melconian manipulan las palabras de...,0,5,20,0,0,5,...,0,0,0,0,0,0,0,0,0,0
25644,Sergio Tomas Massa,youtube,2023-10-08,javier milei la rompe en la mesa de mirtha leg...,0,2,48,0,0,2,...,0,0,0,0,0,0,0,0,0,0


In [50]:
# pandarallel apply function
df_temp = df['text'].parallel_apply(lambda x: pd.Series(LIWC.liwc().getLIWCCount(x)))
df_temp.columns = [f'liwc_{col}' for col in df_temp.columns]

# Directly assign to columns in original dataframe
df[df_temp.columns] = df_temp
print('Done!')
df

/tmp/ipykernel_64769/2247866398.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[df_temp.columns] = df_temp
/tmp/ipykernel_64769/2247866398.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[df_temp.columns] = df_temp
/tmp/ipykernel_64769/2247866398.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame

Done!


,candidate,source,date,text,mentions_myriam teresa bregma,mentions_myriam bregman,mentions_miriam bregman,mentions_miriam breman,mentions_myriam breman,mentions_myriam,...,liwc_Trabajo,liwc_Logro,liwc_Placer,liwc_Hogar,liwc_Dinero,liwc_Relig,liwc_Muerte,liwc_Asentir,liwc_NoFluen,liwc_Relleno
0,Alejandro Biondini,c5n,2023-06-27,los precandidatos presidenciales firmarán un c...,0,2,0,0,0,2,...,6,4,7,1,0,2,0,0,0,0
1,Alejandro Biondini,c5n,2023-06-29,la interna menos pensada: el verborrágico sant...,0,0,0,0,0,0,...,6,6,5,1,0,1,0,0,0,0
2,Alejandro Biondini,c5n,2023-09-05,"quién es lucía montenegro, una de las anfitrio...",0,0,0,0,0,0,...,10,7,4,1,1,2,0,1,0,0
3,Alejandro Biondini,clarin,2017-10-09,"por la foto con biondini, ricardo iorio fue da...",0,0,0,0,0,0,...,7,4,14,2,6,0,2,1,0,0
4,Alejandro Biondini,clarin,2019-08-09,googleó su nombre y descubrió que el domingo e...,0,0,0,0,0,0,...,11,3,10,1,3,0,1,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25641,Sergio Tomas Massa,youtube,2023-10-05,rompecabezas - patricia bullrich pugna por ent...,0,9,31,0,0,9,...,29411,21314,10951,3630,14702,3356,1880,9617,5632,5
25642,Sergio Tomas Massa,youtube,2023-10-06,el poder económico entre milei y bullrich: jai...,0,1,33,0,0,1,...,16383,11418,5826,1796,8334,1304,857,5194,5080,7
25643,Sergio Tomas Massa,youtube,2023-10-07,bullrich y melconian manipulan las palabras de...,0,5,20,0,0,5,...,8817,6606,3089,1064,4348,1033,498,2919,2416,1
25644,Sergio Tomas Massa,youtube,2023-10-08,javier milei la rompe en la mesa de mirtha leg...,0,2,48,0,0,2,...,8068,6118,3049,1070,3500,863,692,3100,2563,3


In [51]:
df_temp = df['text'].parallel_apply(lambda x: pd.Series(anew_extract(x,anew_dict)))
df_temp.columns = [f'anew_{col}' for col in df_temp.columns]

# Directly assign to columns in original dataframe
df[df_temp.columns] = df_temp
print('Done!')
df

/tmp/ipykernel_64769/2138863310.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[df_temp.columns] = df_temp
/tmp/ipykernel_64769/2138863310.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[df_temp.columns] = df_temp


Done!


,candidate,source,date,text,mentions_myriam teresa bregma,mentions_myriam bregman,mentions_miriam bregman,mentions_miriam breman,mentions_myriam breman,mentions_myriam,...,liwc_Placer,liwc_Hogar,liwc_Dinero,liwc_Relig,liwc_Muerte,liwc_Asentir,liwc_NoFluen,liwc_Relleno,anew_valence,anew_arousal
0,Alejandro Biondini,c5n,2023-06-27,los precandidatos presidenciales firmarán un c...,0,2,0,0,0,2,...,7,1,0,2,0,0,0,0,5.640250,5.436500
1,Alejandro Biondini,c5n,2023-06-29,la interna menos pensada: el verborrágico sant...,0,0,0,0,0,0,...,5,1,0,1,0,0,0,0,5.306863,5.189020
2,Alejandro Biondini,c5n,2023-09-05,"quién es lucía montenegro, una de las anfitrio...",0,0,0,0,0,0,...,4,1,1,2,0,1,0,0,5.622239,5.047463
3,Alejandro Biondini,clarin,2017-10-09,"por la foto con biondini, ricardo iorio fue da...",0,0,0,0,0,0,...,14,2,6,0,2,1,0,0,5.594854,5.102233
4,Alejandro Biondini,clarin,2019-08-09,googleó su nombre y descubrió que el domingo e...,0,0,0,0,0,0,...,10,1,3,0,1,2,0,0,5.350376,5.152481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25641,Sergio Tomas Massa,youtube,2023-10-05,rompecabezas - patricia bullrich pugna por ent...,0,9,31,0,0,9,...,10951,3630,14702,3356,1880,9617,5632,5,5.571718,5.004509
25642,Sergio Tomas Massa,youtube,2023-10-06,el poder económico entre milei y bullrich: jai...,0,1,33,0,0,1,...,5826,1796,8334,1304,857,5194,5080,7,5.566484,5.003138
25643,Sergio Tomas Massa,youtube,2023-10-07,bullrich y melconian manipulan las palabras de...,0,5,20,0,0,5,...,3089,1064,4348,1033,498,2919,2416,1,5.529086,5.022642
25644,Sergio Tomas Massa,youtube,2023-10-08,javier milei la rompe en la mesa de mirtha leg...,0,2,48,0,0,2,...,3049,1070,3500,863,692,3100,2563,3,5.590878,5.009874


In [52]:
df_temp = df['text'].parallel_apply(lambda x: pd.Series(readability_index(x)))
df_temp.columns = [f'readability_{col}' for col in df_temp.columns]

# Directly assign to columns in original dataframe
df[df_temp.columns] = df_temp
print('Done!')
df

/tmp/ipykernel_64769/1880088375.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[df_temp.columns] = df_temp
/tmp/ipykernel_64769/1880088375.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[df_temp.columns] = df_temp
/tmp/ipykernel_64769/1880088375.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame

Done!


,candidate,source,date,text,mentions_myriam teresa bregma,mentions_myriam bregman,mentions_miriam bregman,mentions_miriam breman,mentions_myriam breman,mentions_myriam,...,anew_valence,anew_arousal,readability_FKI,readability_FKR,readability_GunningFog,readability_CLI,readability_DaleChall,readability_ARI,readability_linsear,readability_spache
0,Alejandro Biondini,c5n,2023-06-27,los precandidatos presidenciales firmarán un c...,0,2,0,0,0,2,...,5.640250,5.436500,15.104849,16.532329,19.667409,13.886486,19.703333,12.417099,14.235294,11.371086
1,Alejandro Biondini,c5n,2023-06-29,la interna menos pensada: el verborrágico sant...,0,0,0,0,0,0,...,5.306863,5.189020,18.228275,7.767797,23.608392,15.391758,20.159244,17.181580,21.307692,12.314594
2,Alejandro Biondini,c5n,2023-09-05,"quién es lucía montenegro, una de las anfitrio...",0,0,0,0,0,0,...,5.622239,5.047463,17.979325,11.174388,24.077637,12.531772,19.833203,15.331382,22.083333,12.444405
3,Alejandro Biondini,clarin,2017-10-09,"por la foto con biondini, ricardo iorio fue da...",0,0,0,0,0,0,...,5.594854,5.102233,14.000842,32.446061,18.548298,9.828230,19.352579,11.217492,16.409091,11.757522
4,Alejandro Biondini,clarin,2019-08-09,googleó su nombre y descubrió que el domingo e...,0,0,0,0,0,0,...,5.350376,5.152481,14.732306,30.260117,19.602251,9.582982,19.552992,11.835176,17.940000,11.970569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25641,Sergio Tomas Massa,youtube,2023-10-05,rompecabezas - patricia bullrich pugna por ent...,0,9,31,0,0,9,...,5.571718,5.004509,1546.497826,-4056.664058,1631.018087,10.171244,219.308804,2028.117162,2843.124642,580.702912
25642,Sergio Tomas Massa,youtube,2023-10-06,el poder económico entre milei y bullrich: jai...,0,1,33,0,0,1,...,5.566484,5.003138,550.983624,-1396.360689,583.076328,9.719529,89.260771,718.239377,996.400178,211.353485
25643,Sergio Tomas Massa,youtube,2023-10-07,bullrich y melconian manipulan las palabras de...,0,5,20,0,0,5,...,5.529086,5.022642,363.759988,-896.931791,386.004421,9.887678,64.848676,471.844105,658.590042,141.857202
25644,Sergio Tomas Massa,youtube,2023-10-08,javier milei la rompe en la mesa de mirtha leg...,0,2,48,0,0,2,...,5.590878,5.009874,1091.642760,-2840.742722,1152.175971,9.797972,159.870033,1429.613607,1985.010274,411.951684


In [53]:
df_temp = df['text'].parallel_apply(lambda x: pd.Series(complexity_index(x)))
df_temp.columns = [f'complexity_{col}' for col in df_temp.columns]

# Directly assign to columns in original dataframe
df[df_temp.columns] = df_temp
print('Done!')
df

/tmp/ipykernel_64769/2059355663.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[df_temp.columns] = df_temp
/tmp/ipykernel_64769/2059355663.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[df_temp.columns] = df_temp
/tmp/ipykernel_64769/2059355663.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame

Done!


,candidate,source,date,text,mentions_myriam teresa bregma,mentions_myriam bregman,mentions_miriam bregman,mentions_miriam breman,mentions_myriam breman,mentions_myriam,...,readability_spache,complexity_hapaxDislogemna,complexity_hapaxLegomena,complexity_brunet,complexity_sichel,complexity_honore,complexity_TTR,complexity_chars,complexity_digits,complexity_whiteSpace
0,Alejandro Biondini,c5n,2023-06-27,los precandidatos presidenciales firmarán un c...,0,2,0,0,0,2,...,11.371086,0.101974,0.427632,1.178178e+06,0.171271,303.298198,0.595395,6.388158,0.014418,0.156025
1,Alejandro Biondini,c5n,2023-06-29,la interna menos pensada: el verborrágico sant...,0,0,0,0,0,0,...,12.314594,0.065868,0.482036,1.992702e+06,0.107843,319.990818,0.610778,6.580838,0.011829,0.151501
2,Alejandro Biondini,c5n,2023-09-05,"quién es lucía montenegro, una de las anfitrio...",0,0,0,0,0,0,...,12.444405,0.062706,0.504950,1.280686e+06,0.101064,321.153243,0.620462,6.359736,0.003633,0.156720
3,Alejandro Biondini,clarin,2017-10-09,"por la foto con biondini, ricardo iorio fue da...",0,0,0,0,0,0,...,11.757522,0.051392,0.447537,8.937998e+06,0.091954,336.995404,0.558887,5.897216,0.004357,0.169208
4,Alejandro Biondini,clarin,2019-08-09,googleó su nombre y descubrió que el domingo e...,0,0,0,0,0,0,...,11.970569,0.054577,0.417254,2.264413e+07,0.102649,342.144192,0.531690,5.890845,0.001195,0.169456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25641,Sergio Tomas Massa,youtube,2023-10-05,rompecabezas - patricia bullrich pugna por ent...,0,9,31,0,0,9,...,580.702912,0.005168,0.015969,2.787312e+39,0.147806,641.425597,0.034962,5.438857,0.003890,0.183862
25642,Sergio Tomas Massa,youtube,2023-10-06,el poder económico entre milei y bullrich: jai...,0,1,33,0,0,1,...,211.353485,0.007613,0.022652,2.001783e+36,0.159165,618.419629,0.047832,5.361571,0.004127,0.186512
25643,Sergio Tomas Massa,youtube,2023-10-07,bullrich y melconian manipulan las palabras de...,0,5,20,0,0,5,...,141.857202,0.009958,0.031344,7.588228e+32,0.157773,594.590797,0.063118,5.396650,0.004085,0.185300
25644,Sergio Tomas Massa,youtube,2023-10-08,javier milei la rompe en la mesa de mirtha leg...,0,2,48,0,0,2,...,411.951684,0.008973,0.027972,8.593396e+31,0.153199,590.131245,0.058572,5.377353,0.003685,0.185965


In [54]:
print("done")

done


In [55]:
def get_polarity(text):
  # for each sentence in text calculate the polarity score and then take the average of all the sentences, return the avg of compound, positive, negative, and neutral scores
    scores = [sia.polarity_scores(s) for s in nltk.sent_tokenize(text)]
    avg_scores = {}
    # avging scores for all the sentences
    for key in scores[0].keys():
        avg_scores[key] = np.mean([score[key] for score in scores])
    return(avg_scores)


df_temp = df['text'].parallel_apply(lambda x: pd.Series(get_polarity(x)))
df_temp.columns = [f'polarity_{col}' for col in df_temp.columns]

# # Directly assign to columns in original dataframe
df[df_temp.columns] = df_temp
print('Done!')
df

/tmp/ipykernel_64769/1803172093.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[df_temp.columns] = df_temp
/tmp/ipykernel_64769/1803172093.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[df_temp.columns] = df_temp
/tmp/ipykernel_64769/1803172093.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = fr

Done!


,candidate,source,date,text,mentions_myriam teresa bregma,mentions_myriam bregman,mentions_miriam bregman,mentions_miriam breman,mentions_myriam breman,mentions_myriam,...,complexity_sichel,complexity_honore,complexity_TTR,complexity_chars,complexity_digits,complexity_whiteSpace,polarity_neg,polarity_neu,polarity_pos,polarity_compound
0,Alejandro Biondini,c5n,2023-06-27,los precandidatos presidenciales firmarán un c...,0,2,0,0,0,2,...,0.171271,303.298198,0.595395,6.388158,0.014418,0.156025,0.027529,0.972471,0.000000,-0.040865
1,Alejandro Biondini,c5n,2023-06-29,la interna menos pensada: el verborrágico sant...,0,0,0,0,0,0,...,0.107843,319.990818,0.610778,6.580838,0.011829,0.151501,0.008000,0.992000,0.000000,-0.007900
2,Alejandro Biondini,c5n,2023-09-05,"quién es lucía montenegro, una de las anfitrio...",0,0,0,0,0,0,...,0.101064,321.153243,0.620462,6.359736,0.003633,0.156720,0.000000,0.995833,0.004167,0.030100
3,Alejandro Biondini,clarin,2017-10-09,"por la foto con biondini, ricardo iorio fue da...",0,0,0,0,0,0,...,0.091954,336.995404,0.558887,5.897216,0.004357,0.169208,0.024818,0.928545,0.046636,0.029609
4,Alejandro Biondini,clarin,2019-08-09,googleó su nombre y descubrió que el domingo e...,0,0,0,0,0,0,...,0.102649,342.144192,0.531690,5.890845,0.001195,0.169456,0.040080,0.959920,0.000000,-0.147976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25641,Sergio Tomas Massa,youtube,2023-10-05,rompecabezas - patricia bullrich pugna por ent...,0,9,31,0,0,9,...,0.147806,641.425597,0.034962,5.438857,0.003890,0.183862,0.049905,0.942983,0.007158,-0.770460
25642,Sergio Tomas Massa,youtube,2023-10-06,el poder económico entre milei y bullrich: jai...,0,1,33,0,0,1,...,0.159165,618.419629,0.047832,5.361571,0.004127,0.186512,0.056444,0.937078,0.006455,-0.395543
25643,Sergio Tomas Massa,youtube,2023-10-07,bullrich y melconian manipulan las palabras de...,0,5,20,0,0,5,...,0.157773,594.590797,0.063118,5.396650,0.004085,0.185300,0.038398,0.946767,0.010612,-0.289633
25644,Sergio Tomas Massa,youtube,2023-10-08,javier milei la rompe en la mesa de mirtha leg...,0,2,48,0,0,2,...,0.153199,590.131245,0.058572,5.377353,0.003685,0.185965,0.044342,0.947130,0.008562,-0.680355


In [56]:
print("calculated")

calculated


In [57]:
df.to_csv('features_extracted.csv', index=False)

In [58]:
print("all done collected")

all done collected
